In [6]:
import gym
import numpy as np

# Define the action space
action_space = gym.spaces.Box(0.0, 1.0, (2,2), np.int32)

# Sample an action
sampled_action = action_space.sample()

print(sampled_action)

[[1 0]
 [1 1]]
